In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [3]:
browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")

In [4]:
# Lưu user
users = []
# Lưu track
playlists = []
# Lưu playlist
tracks = []
# Lưu link playlist của producer
urls_user_playlist = []
# Lưu link playlist của discover
urls_playlist = []

**Chuyển từ list sang csv/tsv**

In [5]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

**Bấm để đồng ý cookies**

In [6]:
def accept_cookies():
    soup = BeautifulSoup(browser.page_source)
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Bấm vào các nút > đến hết ở trang discover**

In [7]:
def click_button():
    soup = BeautifulSoup(browser.page_source)
    genres = soup.find_all("li", "mixedModularHome__item")

    for i in range(len(genres)):
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        j = 4
        while True:
            if j > len(lists):
                break
            browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
            j += 4

**Lấy hết tất cả playlists ở trang discover**

In [8]:
def get_all_playlists_discover():
    soup = BeautifulSoup(browser.page_source)  
    genres = soup.find_all("li", "mixedModularHome__item")

    for i in range(len(genres)):
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        for pl in lists:
            title = pl.find("a", "playableTile__heading").get_text().strip()
            link = pl.find("a", "playableTile__artworkLink")["href"]
            if title not in playlists:
                playlists.append(title)
                urls_playlist.append("https://soundcloud.com" + link)

**Kéo xuống đến hết trang**

In [9]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

**Lấy tất cả users và tracks của một playlist**

In [24]:
def get_all_users_tracks_playlist():
    soup = BeautifulSoup(browser.page_source)
    
    count = int(soup.find("div", "genericTrackCount__title").get_text())
    search = soup.find_all("li", "systemPlaylistTrackList__item")
    
    isUserPlaylist = False
    if len(search) == 0:
        isUserPlaylist = True
    
    if isUserPlaylist == False:
        if count > 20:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
        search = soup.find_all("li", "systemPlaylistTrackList__item")
            
    elif isUserPlaylist == True:
        if count > 15:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
        search = soup.find_all("li", "trackList__item")

    # Trích xuất user và track trong 1 playlist
    for producer in search:
        user = producer.find("a", "trackItem__username").get_text()
        url = producer.find("a", "trackItem__username").get('href')
        track = producer.find("a", "trackItem__trackTitle").get_text()
        if user not in users:
            users.append(user)
            urls_user_playlist.append("https://soundcloud.com" + url + "/sets")
        if track not in tracks:
            tracks.append(track)

**Lấy tất cả playlists của một user**

In [11]:
def get_all_playlists_user():
    soup = BeautifulSoup(browser.page_source)
    
    posts = soup.find_all("li", "soundList__item")
    
    # kiểm tra có playlist hay không, nếu có chạy dưới, nếu không bỏ qua
    if len(posts) != 0:
        if len(posts) > 4:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
        
        for post in posts:
            title = post.find("a", "soundTitle__title").get_text().strip
            if title not in playlists:
                playlists.append(title)

In [12]:
def parse_users_tracks_per_genre():
    for url in urls_playlist:
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_users_tracks_playlist()
        time.sleep(1)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [13]:
def parse_playlists_per_genre():
    for url in urls_user_playlist:
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_playlists_user()
        time.sleep(1)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [14]:
accept_cookies()

In [15]:
click_button()
get_all_playlists_discover()

In [16]:
parse_users_tracks_per_genre()

In [25]:
parse_playlists_per_genre()

KeyboardInterrupt: 

In [ ]:
browser.quit()

**Lưu data vào file tsv**

In [17]:
outputFile = "Crawl_data/user.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist.csv"
list_to_tsv_csv(playlists, outputFile)

In [26]:
print(len(users))
print(len(tracks))
print(len(playlists))

1281
1852
331
